In [64]:
# coding: utf-8

# In[154]:
import itertools
from datetime import timedelta
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait


# get_ipython().magic(u'matplotlib inline')


def fix_str(s):
    s = s.strip()
    if ':' in s:
        if 'OT' in s: s = s[:s.find('OT')].strip()  # Used for basketball model
        s = s.split(':')
        s = [int(_) for _ in s]
        s = [str(_) for _ in s]
        s = ' '.join(s)


def fix_odds(s):
    try:
        if '+' in s:
            return str(float(s[1:]) / 100 + 1)
        elif '-' in s:
            return str((float(s[1:]) + 100) / float(s[1:]))
        elif '/' in s:
            s = s.strip()
            return str((float(s[:s.find('/')]) / float(s[s.find('/') + 1:]) + 1.0))
        elif '.' in s:
            return s
        else:
            print s
            return 1.0
    except:
        return 1.0


class AcquireMatchupDatetimeOddsTwoChoices(object):
    # -------------------------------------------------------------------------------------------------------------------------------------

    def __init__(self, betbrain_team_names_list_url, betbrain_upcoming_games_url, cs_team_stats_filename, league_name,
                 tableau_filename, upcoming_games_output_filename_us, upcoming_games_output_filename_eu):
        self.betbrain_team_names_list_url = betbrain_team_names_list_url
        self.betbrain_upcoming_games_url = betbrain_upcoming_games_url
        self.cs_team_stats_filename = cs_team_stats_filename
        self.league_name = league_name
        self.tableau_filename = tableau_filename
        self.upcoming_games_output_filename_us = upcoming_games_output_filename_us
        self.upcoming_games_output_filename_eu = upcoming_games_output_filename_eu

    def __call__(self):
        self.get_team_names_list(self.betbrain_team_names_list_url)
        self.generate_data(self.betbrain_upcoming_games_url)
        self.clean_data(self.game_df,  self.league_name)
        self.merge_with_tableau_output(self.game_df, self.tableau_filename, self.upcoming_games_output_filename_us,
                                       self.upcoming_games_output_filename_eu)

    # -------------------------------------------------------------------------------------------------------------------------------------

    # -------------------------------------------------------------------------------------------------------------------------------------

    def get_team_names_list(self, betbrain_team_names_list_url):

        delay = 5

        while True:
            try:
                browser = webdriver.Chrome("C:\Users\jbadiabo\chromedriver.exe")
                browser.get(betbrain_team_names_list_url)
                WebDriverWait(browser, delay).until(ec.presence_of_element_located((By.CLASS_NAME, "DynamicPage")))
            except TimeoutException:
                browser.quit()
                delay += 3
                continue
            break

        browser.maximize_window()
        teams = browser.find_elements_by_class_name("CollapsibleTitle")
        team_names_list = sorted([team.text.encode('utf-8') for team in teams])
        self.team_names_list = team_names_list
        browser.quit()

    # -------------------------------------------------------------------------------------------------------------------------------------

    def generate_data(self, betbrain_upcoming_games_url):

        delay = 5

        while True:
            try:
                browser = webdriver.Chrome("C:\Users\jbadiabo\chromedriver.exe")
                browser.get(betbrain_upcoming_games_url)
                WebDriverWait(browser, delay).until(ec.presence_of_element_located((By.CLASS_NAME, "MatchesList")))
                table_check = browser.find_element_by_class_name("MatchesList")
                body_rows_check = table_check.find_elements_by_class_name("Match")
                if len(body_rows_check) == 0:
                    browser.quit()
                    print "No upcoming matchup for this league. Probably the season is over or on a break..."
                    raise SystemExit(0)  # could use sys.exit()
                else:
                    pass
            except TimeoutException:
                browser.quit()
                delay += 3
                continue
            break

        browser.maximize_window()
        table = browser.find_element_by_class_name("MatchesList")
        body_rows = table.find_elements_by_class_name("Match")

        file_dates = []
        file_matchups = []
        file_odds = []

        for row in body_rows:
            date = row.find_element_by_class_name('DateTime')
            date = [date.text.encode('utf-8')]
            file_dates.append(date)

            matchup = row.find_element_by_class_name('MatchDetails')
            matchup_d1 = matchup.find_element_by_class_name('MatchTitleLink')
            matchup_d1 = [matchup_d1.text.encode('utf-8')]
            file_matchups.append(matchup_d1)

            odds_list = row.find_element_by_class_name('BetList')
            odds = odds_list.find_elements_by_class_name('Bet')
            file_odds_xy = []
            for odd in odds:
                odd_w = odd.find_elements_by_class_name("AverageOdds")
                odd_x = [fix_odds(x.text.encode('ascii', 'ignore')) for x in odd_w]
                file_odds_xy.append(odd_x)
                file_odds_xy_chain = list(itertools.chain.from_iterable(file_odds_xy))
                file_odds_xy_chain = ' '.join(file_odds_xy_chain)

            file_odds.append(file_odds_xy_chain)

        file_matchups = list(itertools.chain.from_iterable(file_matchups))

        browser.quit()

        game_df = pd.DataFrame(file_dates, columns=['Matchup_Date'])

        game_df['Matchup'] = file_matchups

        for date in game_df['Matchup_Date']:
            try:
                date = pd.to_datetime(date, format='%d/%m/%Y %H:%M')
            except ValueError:
                print "in-play game"
                continue

        game_df['Odds'] = file_odds
        game_df['Visitor_Odd'] = game_df["Odds"].str.split(' ').str.get(1)
        game_df['Home_Odd'] = game_df["Odds"].str.split(' ').str.get(0)
        game_df.drop('Odds', axis=1, inplace=True)

        True_Result = []
        string_to_check = "/"
        # For each row in the column,
        for row in game_df['Matchup_Date']:
            # if more than a value,
            if string_to_check in row:
                # Append a letter grade
                True_Result.append('Upcoming')

            else:
                # Append a failing grade
                True_Result.append('IN-PLAY')

        # Create a column from the list
        game_df['True_Result_U'] = True_Result

        # Drop potential matchups in in-play and dropping the Score column for simplicity
        game_df = game_df[game_df.True_Result_U != "IN-PLAY"]
        game_df.drop('True_Result_U', axis=1, inplace=True)

        # CHANGE 'HOURS' AND THE OPERATOR ACCORDING THE TIMEZONE FORMAT FOR MATCHUPS ON THE DATABASE SOURCE WEBSITE
        # eg: see timezone on basketball-reference
        game_df['Matchup_Date_GMT_Minus_4'] = game_df['Matchup_Date']
        game_df['Matchup_Date_GMT_Minus_4'] = pd.to_datetime(game_df['Matchup_Date_GMT_Minus_4'],
                                                             format='%d/%m/%Y %H:%M')
        game_df['Matchup_Date_GMT_Minus_4'] = game_df['Matchup_Date_GMT_Minus_4'] - timedelta(hours=4)
        game_df["Matchup_Date"] = game_df['Matchup_Date_GMT_Minus_4'].astype(str)
        game_df['Date'] = game_df["Matchup_Date"].str.split(' ').str.get(0)
        game_df['Date'] = pd.to_datetime(game_df['Date'], format='%Y/%m/%d')
        game_df['Time'] = game_df["Matchup_Date"].str.split(' ').str.get(1)
        game_df.drop('Matchup_Date', axis=1, inplace=True)
        game_df['Visitor_Team'] = game_df["Matchup"].str.split(' — ').str.get(1)
        game_df['Home_Team'] = game_df["Matchup"].str.split(' — ').str.get(0)
        game_df.drop('Matchup', axis=1, inplace=True)

        self.game_df = game_df

    # -------------------------------------------------------------------------------------------------------------------------------------

    def clean_data(self, game_df, cs_team_stats_filename, team_names_list, league_name):
        # game_df['Visitor_Odd'] = game_df['Visitor_Odd'].str.lstrip('Away\n')
        # game_df['Home_Odd'] = game_df['Home_Odd'].str.lstrip('Home\n')
        game_df['Visitor_Odd'] = game_df['Visitor_Odd'].str.lstrip('(')
        game_df['Visitor_Odd'] = game_df['Visitor_Odd'].str.rstrip(')')
        game_df['Home_Odd'] = game_df['Home_Odd'].str.lstrip('(')
        game_df['Home_Odd'] = game_df['Home_Odd'].str.rstrip(')')


        game_df = game_df[game_df['Visitor_Odd'].notnull()]
        # game_df = game_df[game_df['Visitor_Team'].notnull()]
        # game_df = game_df[game_df['Home_Team'].notnull()]
        self.game_df1 = game_df
        game_df = game_df[game_df['Visitor_Odd'].str.contains('\n') == False]
        game_df = game_df[game_df['Home_Odd'].str.contains('\n') == False]

        game_df['Visitor_Odd'] = pd.to_numeric(game_df['Visitor_Odd'])
        game_df['Home_Odd'] = pd.to_numeric(game_df['Home_Odd'])

        game_df.insert(3, 'League', str(league_name))
        
        # Taking the current season team stats df to take the list of teams
        cs_team_stats_df = pd.read_csv(cs_team_stats_filename)
        db_team_names_list = sorted(cs_team_stats_df['Team'].tolist())

        for s, t in zip(db_team_names_list, team_names_list):
            game_df['Visitor_Team'] = game_df['Visitor_Team'].str.strip().str.replace(t, s)
            game_df['Home_Team'] = game_df['Home_Team'].str.strip().str.replace(t, s)

        self.game_df = game_df
        self.game_df2 = game_df

    # -------------------------------------------------------------------------------------------------------------------------------------

    def merge_with_tableau_output(self, game_df, tableau_filename, upcoming_games_output_filename_us,
                                  upcoming_games_output_filename_eu):

        left = pd.read_csv(tableau_filename)
        left['Visitor_Team'] = left['Visitor_Team']
        left['Home_Team'] = left['Home_Team']
        left['Date'] = pd.to_datetime(left['Date'], infer_datetime_format=True)
        self.left = left

        df = pd.merge(left, game_df, on=['Visitor_Team', 'Home_Team', 'Date'], how="inner")
        self.df = df
        df = df.sort_values(['Date', 'Time'])
        df['Sibyl'] = df.apply(lambda x: x['Home_Team'] if x['Predicted_Result'] == 1 else x['Visitor_Team'], axis=1)
        df['Bookies_choice'] = df.apply(
            lambda x: x['Home_Team'] if x['Home_Odd'] < x['Visitor_Odd'] else x['Visitor_Team'], axis=1)
        df['Divergence_Y/N'] = df[['Sibyl', 'Bookies_choice']].apply(lambda x: 'Y' if x[0] != x[1] else "N", axis=1)

        self.df1 = df
        df = df.drop('ID', axis=1)
        df = df.drop('V_Team_PTS', axis=1)
        df = df.drop('H_Team_PTS', axis=1)
        df['True_Result'] = 'Upcoming'
    
        self.df2 = df
        df['Confidence'] = df['Confidence'].round(3)
        df['Matchup_US_P'] = df['Visitor_Team'] + ' @ ' + df['Home_Team']
        df['Matchup_EU_P'] = df['Home_Team'] + ' vs ' + df['Visitor_Team']

        nba_us_p_df = pd.concat(
            [df['Date'], df['Time'], df['League'], df['Matchup_US_P'], df['Visitor_Team'], df['Home_Team'],
             df['Visitor_Odd'], df['Home_Odd'], df['Bookies_choice'], df['Sibyl'], df['Confidence'], df['Divergence_Y/N'],
             df['True_Result']], axis=1)

        nba_eu_p_df = pd.concat(
            [df['Date'], df['Time'], df['League'], df['Matchup_EU_P'], df['Home_Team'], df['Visitor_Team'],
             df['Home_Odd'], df['Visitor_Odd'], df['Bookies_choice'], df['Sibyl'], df['Confidence'], df['Divergence_Y/N'],
             df['True_Result']], axis=1)

        nba_us_p_df = nba_us_p_df.reset_index(drop=True)
        nba_eu_p_df = nba_eu_p_df.reset_index(drop=True)

        self.nba_us_p_df = nba_us_p_df
        self.nba_eu_p_df = nba_eu_p_df

        nba_us_p_df.to_csv(upcoming_games_output_filename_us, mode='w+', index=True, index_label='ID')
        nba_eu_p_df.to_csv(upcoming_games_output_filename_eu, mode='w+', index=True, index_label='ID')

        # ex of use
x = AcquireMatchupDatetimeOddsTwoChoices(
"https://www.betbrain.com/basketball/united-states/nba/nba/#/winner/whole-event/",\
"https://www.betbrain.com/basketball/united-states/nba/",\
"nba_team_stats_2017_2017.csv",\
"NBA",\
"nba_tableau_output_2017.csv",\
"nba_upcoming_matchups_us_p_df.csv",\
"nba_upcoming_matchups_eu_p_df.csv")
x()

In [65]:
x.nba_us_p_df

,Date,Time,League,Matchup_US_P,Visitor_Team,Home_Team,Visitor_Odd,Home_Odd,Bookies_choice,Sibyl,Confidence,Divergence_Y/N,True_Result
0,2016-12-19,20:00:00,NBA,Washington Wizards @ Indiana Pacers,Washington Wizards,Indiana Pacers,2.85,1.44,Indiana Pacers,Indiana Pacers,0.527,N,Upcoming
1,2016-12-19,21:00:00,NBA,Detroit Pistons @ Chicago Bulls,Detroit Pistons,Chicago Bulls,2.21,1.69,Chicago Bulls,Chicago Bulls,0.678,N,Upcoming
2,2016-12-19,21:00:00,NBA,Phoenix Suns @ Minnesota Timberwolves,Phoenix Suns,Minnesota Timberwolves,2.92,1.42,Minnesota Timberwolves,Minnesota Timberwolves,0.694,N,Upcoming
3,2016-12-19,21:00:00,NBA,Atlanta Hawks @ Oklahoma City Thunder,Atlanta Hawks,Oklahoma City Thunder,2.26,1.66,Oklahoma City Thunder,Oklahoma City Thunder,0.505,N,Upcoming
4,2016-12-19,22:00:00,NBA,Dallas Mavericks @ Denver Nuggets,Dallas Mavericks,Denver Nuggets,4.11,1.24,Denver Nuggets,Denver Nuggets,0.743,N,Upcoming


In [60]:
x.left

,ID,Visitor_Team,V_Team_PTS,Home_Team,H_Team_PTS,True_Result,Predicted_Result,Confidence,Date
0,0,New York Knicks,88,Cleveland Cavaliers,117,1,1,0.891176,2016-10-25
1,1,San Antonio Spurs,129,Golden State Warriors,100,0,1,0.530738,2016-10-25
2,2,Utah Jazz,104,Portland Trail Blazers,113,1,0,0.602504,2016-10-25
3,3,Brooklyn Nets,117,Boston Celtics,122,1,1,0.645854,2016-10-26
4,4,Dallas Mavericks,121,Indiana Pacers,130,1,1,0.638688,2016-10-26
5,5,Houston Rockets,114,Los Angeles Lakers,120,1,1,0.616297,2016-10-26
6,6,Minnesota Timberwolves,98,Memphis Grizzlies,102,1,1,0.510491,2016-10-26
7,7,Charlotte Hornets,107,Milwaukee Bucks,96,0,0,0.609893,2016-10-26
8,8,Denver Nuggets,107,New Orleans Pelicans,102,0,0,0.590569,2016-10-26
9,9,Miami Heat,108,Orlando Magic,96,0,0,0.777214,2016-10-26


In [58]:
x.left

,ID,Visitor_Team,V_Team_PTS,Home_Team,H_Team_PTS,True_Result,Predicted_Result,Confidence,Date
0,0,New York Knicks,88,Cleveland Cavaliers,117,1,1,0.891176,2016-10-25
1,1,San Antonio Spurs,129,Golden State Warriors,100,0,1,0.530738,2016-10-25
2,2,Utah Jazz,104,Portland Trail Blazers,113,1,0,0.602504,2016-10-25
3,3,Brooklyn Nets,117,Boston Celtics,122,1,1,0.645854,2016-10-26
4,4,Dallas Mavericks,121,Indiana Pacers,130,1,1,0.638688,2016-10-26
5,5,Houston Rockets,114,Los Angeles Lakers,120,1,1,0.616297,2016-10-26
6,6,Minnesota Timberwolves,98,Memphis Grizzlies,102,1,1,0.510491,2016-10-26
7,7,Charlotte Hornets,107,Milwaukee Bucks,96,0,0,0.609893,2016-10-26
8,8,Denver Nuggets,107,New Orleans Pelicans,102,0,0,0.590569,2016-10-26
9,9,Miami Heat,108,Orlando Magic,96,0,0,0.777214,2016-10-26
